# Parte 1 🥇

# Baixa corpus ⬇⬇⬇

In [ ]:
import os
import zipfile
import pandas as pd

data_path = "data"
if not os.path.exists(data_path):
  # Baixa arquivo corpus.zip
  url = "https://raw.githubusercontent.com/AngryLeaderBB/NLP_class_2024/refs/" \
        "heads/main/Atividade_3/csvs.zip"
  cmd = ! wget {url}

  # Extrai corpus.zip na pasta corpus
  with zipfile.ZipFile("csvs.zip", 'r') as zip_ref:
      zip_ref.extractall(".")

  # Renomea a pasta csvs para data
  os.rename("csvs", data_path)

  # Exclui arquivo corpus.zip
  os.remove("csvs.zip")

csv_path = data_path + "/re8.csv"

# Carrega o csv no path csv_path
data = pd.read_csv(csv_path)

Imprime parte do corpus

In [ ]:
print(f'file names:\n{data["file_name"]}\n----------')
print(f'texts:\n{data["text"]}\n----------')
print(f'classes:\n{data["class"]}')

file names:
0         acq.4342.txt
1         acq.5302.txt
2         acq.8530.txt
3         acq.3841.txt
4         acq.6302.txt
             ...      
7669    trade.1829.txt
7670    trade.4403.txt
7671    trade.6944.txt
7672    trade.7420.txt
7673    trade.5204.txt
Name: file_name, Length: 7674, dtype: object
----------
texts:
0       mcdowell me to merger with interpharm inc mcdo...
1       intermagnetics general inma completes buy inte...
2       tesco extends hillards offer tesco plc tsco l ...
3       healthvest hvt sells shares healthvest a maryl...
4       cooper canada said it received takeover offers...
                              ...                        
7669    japan will not retaliate now against u s tarif...
7670    koreans to buy billion dlrs in u s goods south...
7671    belgolux trade swings into january deficit the...
7672    nakasone to visit washington in late april pri...
7673    baldrige warns of world trade war danger u s c...
Name: text, Length: 7674, dtype: o

# Baixa arquivo .py

In [ ]:
import os

# Baixa o arquivo find_best_hyperparameters.py
if not os.path.exists("find_best_hyperparameters.py"):
  url = "https://raw.githubusercontent.com/AngryLeaderBB/NLP_class_2024/refs/" \
        "heads/main/Atividade_3/find_best_hyperparameters.py"
  cmd = ! wget {url}


# Divide o corpus em treino e teste ➗

In [ ]:
import importlib
import os
import find_best_hyperparameters as fbh

importlib.reload(fbh)

# Initializa a classe BestParam com o path do csv e
# o path para salvar os resultados
bp = fbh.BestParam(csv_path, data_path)

# Divide os dados de treino/teste e salva no data_path
seed = 12345
bp.split_data(seed)

print("The data has been splited")

The data has been splited


Treina (Opcional, na próxima seção pode-se baixar resultados pré-treinados)

In [ ]:
# Treina modelo e salva
bp.greedy_search(5)
print("The models has been trained")

Fitting 5 folds for each of 14 candidates, totalling 70 fits
Fitting 5 folds for each of 15 candidates, totalling 75 fits
Fitting 5 folds for each of 15 candidates, totalling 75 fits
The models has been trained


# Parte 2 🥈

# Importa de dados base 🎲

In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix

# Carregar os dados de treino e teste
text_train = csr_matrix(pd.read_csv(data_path + "/text_train.csv"))
class_train = pd.read_csv(data_path + "/class_train.csv").squeeze()

text_test = csr_matrix(pd.read_csv(data_path + "/text_test.csv"))
class_test = pd.read_csv(data_path + "/class_test.csv").squeeze()

Carrega os Resultados do Greedy Search

In [ ]:
# Carregar os resultados do Greedy Search
if not os.path.exists(data_path + "/model_search_results.csv"):
  url = "https://raw.githubusercontent.com/AngryLeaderBB/NLP_class_2024/refs/" \
        "heads/main/Atividade_3/model_search_results2.csv"

  cmd = ! wget {url}

  os.rename("model_search_results2.csv", "model_search_results.csv")

  !mv model_search_results.csv {data_path}

search_results = pd.read_csv(data_path + "/model_search_results.csv")

# Exibi os resultados do Greedy Search
print(search_results)

                 Model                                 Params  Mean Test Score
0   LogisticRegression       {'C': 10, 'solver': 'liblinear'}         0.970190
1   LogisticRegression        {'C': 1, 'solver': 'liblinear'}         0.969213
2   LogisticRegression      {'C': 100, 'solver': 'liblinear'}         0.968887
3   LogisticRegression           {'C': 10, 'solver': 'lbfgs'}         0.967421
4   LogisticRegression          {'C': 100, 'solver': 'lbfgs'}         0.967258
5   LogisticRegression            {'C': 1, 'solver': 'lbfgs'}         0.965303
6   LogisticRegression      {'C': 0.1, 'solver': 'liblinear'}         0.963511
7   LogisticRegression            {'C': 10, 'solver': 'saga'}         0.961719
8   LogisticRegression             {'C': 1, 'solver': 'saga'}         0.961556
9   LogisticRegression           {'C': 0.1, 'solver': 'saga'}         0.961393
10  LogisticRegression           {'C': 100, 'solver': 'saga'}         0.961230
11  LogisticRegression          {'C': 0.1, 'solver':

# Re-treina os Modelos 🤖

Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Recupera o melhor parametro com modelo MultinomialNB
best_params_arr = search_results.loc[
                    search_results['Model'] == 'MultinomialNB'].values[0]
print(best_params_arr)

best_params = eval(best_params_arr[1])

# Inicializa o modelo
model_nb = MultinomialNB(alpha=best_params["alpha"],
                         fit_prior=best_params["fit_prior"])

# Treina o modelo
model_nb.fit(text_train, class_train)

['MultinomialNB' "{'alpha': 0.1, 'fit_prior': True}" 0.9400550586823612]


MultinomialNB(alpha=0.1)

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Recupera o melhor parametro com modelo LogisticRegression
best_params_arr = search_results.loc[
                      search_results['Model'] == 'LogisticRegression'].values[0]
print(best_params_arr)

best_params = eval(best_params_arr[1])

# Inicializa o modelo
model_lr = LogisticRegression(C=best_params["C"],
                              solver=best_params["solver"])

# Treina o modelo
model_lr.fit(text_train, class_train)

['LogisticRegression' "{'C': 10, 'solver': 'liblinear'}"
 0.9701902630551992]


LogisticRegression(C=10, solver='liblinear')

Support Vector Classification

In [ ]:
from sklearn.svm import SVC

# Recupera o melhor parametro com modelo SVC
best_params_arr = search_results.loc[
                      search_results['Model'] == 'SVC'].values[0]
print(best_params_arr)

best_params = eval(best_params_arr[1])

# Inicializa o modelo
model_svc = SVC(C=best_params["C"],
                kernel=best_params["kernel"])

# Treina o modelo
model_svc.fit(text_train, class_train)

['SVC' "{'C': 0.1, 'kernel': 'linear'}" 0.9582996848859404]


# Calcula métricas ⭕

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Predições com os modelos treinados
class_pred_nb = model_nb.predict(text_test)
class_pred_lr = model_lr.predict(text_test)
class_pred_svc = model_svc.predict(text_test)

# Calcular as métricas para o Multinomial Naive Bayes
accuracy_nb = accuracy_score(class_test, class_pred_nb)
f1_nb_macro = f1_score(class_test, class_pred_nb, average='macro')
f1_nb_micro = f1_score(class_test, class_pred_nb, average='micro')

# Calcular as métricas para o Logistic Regression
accuracy_lr = accuracy_score(class_test, class_pred_lr)
f1_lr_macro = f1_score(class_test, class_pred_lr, average='macro')
f1_lr_micro = f1_score(class_test, class_pred_lr, average='micro')

# Calcular as métricas para o Support Vector Classification
accuracy_svc = accuracy_score(class_test, class_pred_svc)
f1_svc_macro = f1_score(class_test, class_pred_svc, average='macro')
f1_svc_micro = f1_score(class_test, class_pred_svc, average='micro')

# Exibir os resultados
print(f"Multinomial Naive Bayes:\n Acurácia: {accuracy_nb}"\
      f"\n F1 (Macro): {f1_nb_macro}\n F1 (Micro): {f1_nb_micro}\n")

print(f"Logistic Regression:\n Acurácia: {accuracy_lr}"\
      f"\n F1 (Macro): {f1_lr_macro}\n F1 (Micro): {f1_lr_micro}\n")

print(f"SVC:\n Acurácia: {accuracy_svc}"\
      f"\n F1 (Macro): {f1_svc_macro}\n F1 (Micro): {f1_svc_micro}")

Multinomial Naive Bayes:
 Acurácia: 0.9348534201954397
 F1 (Macro): 0.8910110039527415
 F1 (Micro): 0.9348534201954397

Logistic Regression:
 Acurácia: 0.9680781758957655
 F1 (Macro): 0.924246246073313
 F1 (Micro): 0.9680781758957655

SVC:
 Acurácia: 0.965472312703583
 F1 (Macro): 0.9308432360697406
 F1 (Micro): 0.965472312703583


# Justificativa 🧠

* **Multinomial Naive Bayes**:  O modelo obteve o pior desempenho dos modelos testados. O parâmetro alpha foi escolhido como 0.1, o que é o valor com menos suavização do que padrão para Naive Bayes, o que é um valor de alpha menor do que na outra base de dados que deu alpha igual a 1. A escolha de fit_prior=True ajuda o modelo a aprender as distribuições de classes (ao invés de assumir probabilidades iguais), o que é útil se as classes forem desbalanceadas.

* **Logistic Regression**: Foi o modelo que obteve o melhor desempenho entre os três. O valor de C=10 reflete uma escolha para dar mais flexibilidade ao modelo, permitindo uma menor regularização para que o modelo se ajuste melhor aos dados. A escolha do solver 'liblinear' é eficiente e adequada para problemas menores com menos variáveis e um número razoável de amostras.

* **SVC**: O modelo teve um bom desempenho, mas ficou atrás da Logistic Regression em termos de acurácia e F1. O valor de C = 0.1 indica uma tentativa de balancear com menor suavização a penalização por erros e a margem do modelo. Usar o kernel linear indica que o modelo foi ajustado para trabalhar com separações lineares, o que pode ser suficiente em problemas de classificação mais simples.